In [ ]:
import numpy as np
import random
from datetime import datetime
import matplotlib.pyplot as plt

n_cities = 20
n_population = 100
mutation_rate = 0.3

#Generación de coordenadas listadas representando cada ciudad
coordinates_list = [[x, y] for x, y in zip(np.random.randint(0, 100, n_cities), np.random.randint(0, 100, n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv',
                       'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = {x: y for x, y in zip(names_list, coordinates_list)}

#Distancia entre 2 puntos
def compute_city_distance_coordinates(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

#Creación del primero conjunto poblacional
def genesis(city_list, n_population):
    population_set = []
    for i in range(n_population):
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

#Evaluación solución fitness
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities - 1):
        a = city_list[i]
        b = city_list[i + 1]
        total += compute_city_distance_names(a, b, cities_dict)
    return total

def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)
    for i in range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)
    return fitnes_list

#Métodos de selección de progenitores
def progenitor_selection(population_set, fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = total_fit / fitnes_list  #Inversión de probabilidades
    prob_list /= prob_list.sum()
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set), p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set), p=prob_list, replace=True)
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    return np.array([progenitor_list_a, progenitor_list_b])

#Mates
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[:5]
    for city in prog_b:
        if city not in offspring:
            offspring = np.concatenate((offspring, [city]))
    return offspring

def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
    return new_population_set

#Mutaciones
def mutate_offspring(offspring):
    for q in range(int(n_cities * mutation_rate)):
        a = np.random.randint(0, n_cities)
        b = np.random.randint(0, n_cities)
        offspring[a], offspring[b] = offspring[b], offspring[a]
    return offspring

def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

#Detención
def run_algorithm(n_iterations, selection_method):
    best_solution = [-1, np.inf, np.array([])]
    history = []
    for i in range(n_iterations):
        fitnes_list = get_all_fitnes(mutated_pop, cities_dict)
        if fitnes_list.min() < best_solution[1]:
            best_solution[0] = i
            best_solution[1] = fitnes_list.min()
            best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
        if selection_method == 'roulette':
            progenitor_list = progenitor_selection_roulette(population_set, fitnes_list)
        elif selection_method == 'tournament':
            progenitor_list = progenitor_selection_tournament(population_set, fitnes_list)
        elif selection_method == 'elite':
            progenitor_list = progenitor_selection_elite(population_set, fitnes_list)
        new_population_set = mate_population(progenitor_list)
        mutated_pop = mutate_population(new_population_set)
        #Guardado del mejor fitness por generación
        history.append(fitnes_list.min())
    return best_solution, history

def plot_history(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history, label='Fitness (distance)')
    plt.xlabel('Generations')
    plt.ylabel('Distance')
    plt.title('TSP Solution Evolution')
    plt.legend()
    plt.grid(True)
    plt.show()

#Ejemplos: 1000 iteraciones por ruleta
best_solution_roulette_1000, history_roulette_1000 = run_algorithm(1000, 'roulette')
plot_history(history_roulette_1000)

#Torneo con 5000 iteraciones
best_solution_tournament_5000, history_tournament_5000 = run_algorithm(5000, 'tournament')
plot_history(history_tournament_5000)

#Elite mediante 10000 itraciones
best_solution_elite_10000, history_elite_10000 = run_algorithm(10000, 'elite')
plot_history(history_elite_10000)